**Snipping License Plate from Video**

In [1]:
import numpy as np
import cv2
import os

cap = cv2.VideoCapture('video.mp4')
plate_cascade = cv2.CascadeClassifier('haarcascade_russian_plate_number.xml')

count = 0;
while(cap.isOpened()==True):
    try:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        plate = plate_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in plate:
            cv2.rectangle(frame, (x,y), (x + w, y + h), (0,0,255), 2)
            cv2.imwrite("./cropped/"+str(count)+".jpg", frame[y:y+h, x:x+w])
            count+=1

        cv2.imshow('img', frame)
        if cv2.waitKey(1)==13:
            break
    except:
        break
cap.release()
cv2.destroyAllWindows()

**Importing Libraries**

In [2]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr

In [3]:
import tensorflow as tf
from sklearn.metrics import f1_score
import requests
import xmltodict
import json
import re
import warnings
warnings.filterwarnings("ignore")

**Function for Extext Text from Number Plate**

In [4]:
def ANPR(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction
    edged = cv2.Canny(bfilter, 30, 200) #Edge detection
    
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
    
    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break
    
    mask = np.zeros(gray.shape, np.uint8)
    
    try:
        new_image = cv2.drawContours(mask, [location], 0,255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)
    except:
        return None
    
    (x,y) = np.where(mask==255)
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = gray[x1:x2+1, y1:y2+1]
    
    reader = easyocr.Reader(['en'])
    result = reader.readtext(cropped_image)

    return result

**Appending Text to list**

In [5]:
from os import listdir
from os.path import isfile, join
import threading
from threading import Thread

data_path = './cropped/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

license_plate_number = []

for i, files in enumerate(onlyfiles):
    img_path = data_path + onlyfiles[i]
    img = cv2.imread(img_path)

    number_plate = ANPR(img)

    if number_plate!=None and len(number_plate)==1:
            if(number_plate[0][-1]>0.99):
                number_plate = number_plate[0][-2].replace(" ", "")
                license_plate_number.append(number_plate)
                print(license_plate_number)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

['MH13AZ1881']


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

**Removing Duplicate Texts**

In [11]:
license_plate_number = set(license_plate_number)
license_plate_number = list(license_plate_number)
license_plate_number

['MH13AZ1881']

In [12]:
license_plate_number[0]

'MH13AZ1881'

**Getting Vehicle Information**

In [13]:

def get_vehicle_info(plate_number):
    r = requests.get("https://www.regcheck.org.uk/api/reg.asmx/CheckIndia?RegistrationNumber={0}&username=rahul_xt_".format(str(plate_number)))
#     print(r.content)
    data = xmltodict.parse(r.content)
    jdata = json.dumps(data)
    df = json.loads(jdata)
    df1 = json.loads(df['Vehicle']['vehicleJson'])
    return df1

In [14]:
import re
def plate_info(numberPlate):
    pattern = '^[A-Z]{2}[0-9]{1,2}([A-Z])?([A-Z]*)?[0-9]{4}$'
    if len(numberPlate) > 10:
        numberPlate = numberPlate[-10:]
        return (get_vehicle_info(numberPlate))
#     else:
#         return get_vehicle_info(numberPlate)
    elif re.match(pattern,numberPlate) != None:
        return (get_vehicle_info(numberPlate))

In [22]:
plate_info(license_plate_number[0])

ExpatError: syntax error: line 1, column 0

In [ ]:
import numpy as np
import cv2
import os

cap = cv2.imread('car2.jpg')
plate_cascade = cv2.CascadeClassifier('haarcascade_russian_plate_number.xml')

count=0
gray = cv2.cvtColor(cap, cv2.COLOR_BGR2GRAY)
plate = plate_cascade.detectMultiScale(gray, 1.3, 5)

for (x, y, w, h) in plate:
    cv2.rectangle(cap, (x,y), (x + w, y + h), (0,0,255), 2)
cv2.imwrite("./cropped/"+str(count)+".jpg", cap[y:y+h, x:x+w])
count+=1

cv2.imshow('img', cap)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.imread('./cropped/0.jpg')
ANPR(cap)

In [ ]:

number_plate = ANPR(img)
number_plate = plate_info(number_plate)
number_plate